In [96]:
import os 
import numpy as np 
import cv2
from glob import glob
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from keras.preprocessing.image import ImageDataGenerator
from albumentations import HorizontalFlip, CoarseDropout, RandomBrightnessContrast, RandomRotate90


In [97]:
os.environ["PYTHONHASHSEED"]= str(42)
np.random.seed(42)
tf.random.set_seed(42)

In [98]:
batch_size=8
lr=0.0001
epoch=100
height=128
width=128

In [99]:
dataset_path = os.path.join("dataset2", "non-aug")
files_dir= os.path.join("files", "non-aug")
model_file=os.path.join(files_dir, "unet-non-aug.h5")
log_file=os.path.join(files_dir, "log-non-aug.csv")

In [100]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

create_dir(files_dir)

In [101]:
def conv_block(inputs, num_filters):
    x= Conv2D(num_filters, 3, padding="same")(inputs)
    x= BatchNormalization()(x)
    x=Activation('relu')(x)

    x=Conv2D(num_filters, 3, padding="same")(x)
    x=BatchNormalization()(x)
    x=Activation("relu")(x)
    return x

In [102]:
def encoder_block(inputs, num_filters):
    x= conv_block(inputs, num_filters)
    p=MaxPool2D((2,2))(x)
    return x,p

In [103]:
def decoder_block(inputs, skip, num_filters):
    x=Conv2DTranspose(num_filters, (2,2), strides=2, padding="same")(inputs)
    x= Concatenate()([x,skip])
    x= conv_block(x,num_filters)
    return x

In [104]:
def build_unet(input_shape):
    inputs =Input(input_shape)
    s1,p1=encoder_block(inputs,64)
    s2,p2=encoder_block(p1, 128)
    s3,p3=encoder_block(p2,512)
    s4,p4=encoder_block(p3,512)
    b1= conv_block(p4, 1024)

    d1=decoder_block(b1,s4,512)
    d2=decoder_block(d1,s3,256)
    d3=decoder_block(d2,s2,128)
    d4=decoder_block(d3,s1,64)
    outputs= Conv2D(1,1, padding="same", activation="sigmoid")(d4)
    model= Model(inputs, outputs, name="UNET")
    return model

In [105]:
def load_data(path):
    train_x= sorted(glob(os.path.join(path, "train", "images", "*" )))
    train_y= sorted(glob(os.path.join(path, "train", "masks", "*" )))
    test = sorted(glob(os.path.join(path, "test", "*" )))
    valid_x= sorted(glob(os.path.join(path, "valid", "images", "*" )))
    valid_y= sorted(glob(os.path.join(path, "valid", "masks", "*" )))

    return (train_x, train_y), (valid_x, valid_y) , test

In [106]:
def read_image(path):
    path=path.decode()
    x=cv2.imread(path, cv2.IMREAD_COLOR)
    x=x/255.0
    return x

In [107]:
def read_mask(path):
    path=path.decode()
    x=cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x=x/255.0
    x=np.expand_dims(x,axis=-1)
    return x

In [108]:
def tf_parse(x,y):
    def _parse(x,y):
        x= read_image(x)
        y= read_mask(y)
        return x,y
    x,y =tf.numpy_function(_parse, [x,y], [tf.float64,tf.float64])
    x.set_shape([height, width, 3])
    y.set_shape([height, width, 1])
    return x,y

In [109]:
def tf_dataset(x,y, batch=8):
    dataset= tf.data.Dataset.from_tensor_slices((x,y))
    dataset= dataset.map(tf_parse,num_parallel_calls=tf.data.AUTOTUNE)
    dataset=dataset.batch(batch)
    dataset= dataset.prefetch(tf.data.AUTOTUNE)
    return dataset



In [110]:
(train_x, train_y), (valid_x, valid_y),test= load_data(dataset_path)

print("train" , len(train_x), len(train_y))
print("valid" , len(valid_x), len(valid_y))
print("test" , len(test))

train 1195 1195
valid 239 239
test 2


In [111]:


train_dataset= tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset= tf_dataset(valid_x, valid_y, batch=batch_size)



In [112]:
input_shape = (height, width, 3)
model= build_unet(input_shape)

In [113]:
model.summary()

opt=tf.keras.optimizers.Adam(lr)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_38 (Conv2D)             (None, 128, 128, 64  1792        ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_36 (BatchN  (None, 128, 128, 64  256        ['conv2d_38[0][0]']              
 ormalization)                  )                                                              

In [114]:
callbacks=[

    ModelCheckpoint(model_file, verbose=1, save_best_only=True), 
    ReduceLROnPlateau(monitor='val_loss', factor=.1, patience=4),
    CSVLogger(log_file), 
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
]

In [115]:
model.fit(
    train_dataset, validation_data=valid_dataset, epochs=100, callbacks=callbacks
)

Epoch 1/100


150/150 [==============================] - ETA: 0s - loss: 0.3456 - accuracy: 0.9315
Epoch 1: val_loss improved from inf to 0.55076, saving model to files/non-aug/unet-non-aug.h5
150/150 [==============================] - 21s 128ms/step - loss: 0.3456 - accuracy: 0.9315 - val_loss: 0.5508 - val_accuracy: 0.8833 - lr: 1.0000e-04
Epoch 2/100
150/150 [==============================] - ETA: 0s - loss: 0.2471 - accuracy: 0.9327
Epoch 2: val_loss improved from 0.55076 to 0.36001, saving model to files/non-aug/unet-non-aug.h5
150/150 [==============================] - 17s 110ms/step - loss: 0.2471 - accuracy: 0.9327 - val_loss: 0.3600 - val_accuracy: 0.8914 - lr: 1.0000e-04
Epoch 3/100
150/150 [==============================] - ETA: 0s - loss: 0.2104 - accuracy: 0.9361
Epoch 3: val_loss improved from 0.36001 to 0.32428, saving model to files/non-aug/unet-non-aug.h5
150/150 [==============================] - 16s 109ms/step - loss: 0.2104 - accuracy: 0.9361 - val_loss: 0.3243 - val_accuracy: 0.

In [116]:

save_path= "prediction,non-aug"


create_dir(save_path)

model2=tf.keras.models.load_model(model_file)
model2.summary()


test_x=sorted(os.path.join(dataset_path, 'test', "*"))

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_38 (Conv2D)             (None, 128, 128, 64  1792        ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_36 (BatchN  (None, 128, 128, 64  256        ['conv2d_38[0][0]']              
 ormalization)                  )                                                              

In [127]:
test_x=sorted(glob(os.path.join("/home/keanuf/.cache/Assignemt9/BRAIN SEGMENTATION/dataset2/non-aug/test", "images","*")))
test_x[0]

'/home/keanuf/.cache/Assignemt9/BRAIN SEGMENTATION/dataset2/non-aug/test/images/0.png'

In [128]:
time_taken=[]
import time
for x in tqdm(test_x):
    name=x.split("/")[-1]
    print(name)
    x= cv2.imread(x ,cv2.IMREAD_COLOR)
    x=x / 255.0
    x=np.expand_dims(x,axis=0)
    start_time=time.time()
    p= model.predict(x)[0]
    total_time=time.time()-start_time
    time_taken.append(total_time)
    p=p>0.5
    p=p*255
    cv2.imwrite(os.path.join(save_path, name),p)

   



  0%|          | 0/239 [00:00<?, ?it/s]

0.png
1/1 [==============================] - 0s 23ms/step


  0%|          | 1/239 [00:00<01:03,  3.75it/s]

1.png
1/1 [==============================] - 0s 26ms/step
10.png
1/1 [==============================] - 0s 22ms/step


  1%|▏         | 3/239 [00:00<00:26,  8.87it/s]

...

 98%|█████████▊| 235/239 [00:16<00:00,  7.80it/s]

96.png
1/1 [==============================] - 0s 32ms/step
97.png
1/1 [==============================] - 0s 32ms/step


 99%|█████████▉| 237/239 [00:16<00:00,  8.63it/s]

98.png
1/1 [==============================] - 0s 60ms/step
99.png
1/1 [==============================] - 0s 36ms/step


100%|██████████| 239/239 [00:16<00:00, 14.18it/s]


In [129]:
import os
from PIL import Image

def composite_images(image_folder, mask_folder, output_folder):
    # Get the list of image files in the input folder
    image_files = os.listdir(image_folder)
    
    # Iterate through each image file
    for image_file in image_files:
        # Construct the file paths for the current image and its corresponding mask
        image_path = os.path.join(image_folder, image_file)
        mask_file = image_file.split('.')[0] + '.png'  # Assuming masks have the same filename format as images
        mask_path = os.path.join(mask_folder, mask_file)
        
        # Open the image and mask
        image = Image.open(image_path)
        mask = Image.open(mask_path)
        
        # Perform composite operation
        result = Image.composite(image, Image.new('RGB', image.size, (0, 0, 0)), mask)
        
        # Construct the output file path
        output_file = os.path.join(output_folder, image_file)
        
        # Save the resulting composite image
        result.save(output_file)

# Paths to input image and mask folders, and output folder
image_folder = '/home/keanuf/.cache/Assignemt9/BRAIN SEGMENTATION/dataset2/non-aug/test/images'
mask_folder = '/home/keanuf/.cache/Assignemt9/BRAIN SEGMENTATION/prediction,non-aug'
output_folder = '/home/keanuf/.cache/BRAIN SEGMENTATOIN'

# Call the function to composite images and masks and save the results
composite_images(image_folder, mask_folder, output_folder)